<a href="https://colab.research.google.com/github/PrathamK03/python-program/blob/main/CART1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import math
import numpy as np

data = pd.read_csv("dataset.csv")
features = [feat for feat in data]
features.remove("answer")

In [ ]:
class Node:
    def __init__(self):
        self.children = []
        self.value = ""
        self.isLeaf = False
        self.pred = ""

In [ ]:
def gini(examples):
    pos = 0.0
    neg = 0.0
    for _, row in examples.iterrows():
        if row["answer"] == "yes":
            pos += 1
        else:
            neg += 1
    if pos == 0.0 or neg == 0.0:
        return 0.0
    else:
       t=pos+neg
       x=pos/t
       y=neg/t
       p=x*x
       n=y*y
       return 1-p-n

In [ ]:
def cart(examples, attrs):
    root = Node()

    min_gini = float('inf')  # Initialize min_gini to a high value
    max_feat = ""
    for feature in attrs:
        gini_index = gini(examples)
        if gini_index < min_gini:
            min_gini = gini_index
            max_feat = feature

    root.value = max_feat
    uniq = np.unique(examples[max_feat])

    for u in uniq:
        subdata = examples[examples[max_feat] == u]
        if gini(subdata) == 0.0:
            newNode = Node()
            newNode.isLeaf = True
            newNode.value = u
            newNode.pred = np.unique(subdata["answer"])
            root.children.append(newNode)
        else:
            dummyNode = Node()
            dummyNode.value = u
            new_attrs = attrs.copy()
            new_attrs.remove(max_feat)
            child = cart(subdata, new_attrs)
            dummyNode.children.append(child)
            root.children.append(dummyNode)

    return root

In [ ]:
def printTree(root: Node, depth=0):
    for i in range(depth):
        print("\t", end="")
    print(root.value, end="")
    if root.isLeaf:
        print(" -> ", root.pred)
    print()
    for child in root.children:
        printTree(child, depth + 1)

In [ ]:
def classify(root: Node, new):
    if not new:  # Check if the new dictionary is empty
        print("Error: The new example dictionary is empty.")
        return None

    for child in root.children:
        if child.value in new:
            if child.value == new[child.value]:
                if child.isLeaf:
                    print("Predicted Label for new example", new, " is:", child.pred)
                    return child.pred
                else:
                    return classify(child, new)
        else:
            print(f"Error: The key '{child.value}' not found in the new example dictionary.")
    print("Error: No matching child found for the new example.")
    return None
    print("Keys in new example dictionary:", new.keys())
    print("Features expected by the decision tree:", features)

In [ ]:
root = cart(data, features)
print("Decision Tree is:")
printTree(root)
print("------------------")

new = {"outlook": "Sunny", "temperature": "Hot", "humidity": "Normal", "wind": "Strong"}

# Print the features list
print("Features:", features)

classify(root, new)

Decision Tree is:
outlook
	overcast ->  ['yes']

	rain
		temperature
			cool
				humidity
					normal
						wind
							strong ->  ['no']

							weak ->  ['yes']

			mild
				humidity
					high
						wind
							strong ->  ['no']

							weak ->  ['yes']

					normal ->  ['yes']

	sunny
		temperature
			cool ->  ['yes']

			hot ->  ['no']

			mild
				humidity
					high ->  ['no']

					normal ->  ['yes']

------------------
Features: ['outlook', 'temperature', 'humidity', 'wind']
Error: The key 'overcast' not found in the new example dictionary.
Error: The key 'rain' not found in the new example dictionary.
Error: The key 'sunny' not found in the new example dictionary.
Error: No matching child found for the new example.
